In [3]:
from huggingface_hub import snapshot_download; snapshot_download(repo_id="ibm-granite/granite-8b-code-instruct-128k", local_dir="granite_model")

Fetching 12 files:   0%|          | 0/12 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.26G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

'/home/ec2-user/SageMaker/aws-samples/bedrock/model_import/granite_model'

In [6]:
!aws s3 cp --recursive granite_model s3://virgilio-murillo/work/model_import/IBMGranite-granite-8b-code-instruct-128k/granite_model/

upload: granite_model/.cache/huggingface/.gitignore to s3://virgilio-murillo/work/model_import/IBMGranite-granite-8b-code-instruct-128k/granite_model/.cache/huggingface/.gitignore
upload: granite_model/.cache/huggingface/download/README.md.lock to s3://virgilio-murillo/work/model_import/IBMGranite-granite-8b-code-instruct-128k/granite_model/.cache/huggingface/download/README.md.lock
upload: granite_model/.cache/huggingface/download/.gitattributes.lock to s3://virgilio-murillo/work/model_import/IBMGranite-granite-8b-code-instruct-128k/granite_model/.cache/huggingface/download/.gitattributes.lock
upload: granite_model/.cache/huggingface/download/.gitattributes.metadata to s3://virgilio-murillo/work/model_import/IBMGranite-granite-8b-code-instruct-128k/granite_model/.cache/huggingface/download/.gitattributes.metadata
upload: granite_model/.cache/huggingface/download/model-00002-of-00004.safetensors.lock to s3://virgilio-murillo/work/model_import/IBMGranite-granite-8b-code-instruct-128k/gr

added a custom IAM policy to allow access:

```
{
	"Version": "2012-10-17",
	"Statement": [
		{
			"Sid": "VisualEditor0",
			"Effect": "Allow",
			"Action": "bedrock:InvokeModel",
			"Resource": "arn:aws:bedrock:us-east-1:794038231401:imported-model/o86ksgjlukpi"
		}
	]
}
```

In [14]:
# Use the native inference API to send a text message to Meta Llama 3.

import boto3
import json

from botocore.exceptions import ClientError
from botocore.config import Config

config = Config(
    retries={
        'total_max_attempts': 10,
        'mode': 'standard'
    }
)

# Create a Bedrock Runtime client in the AWS Region of your choice.
client = boto3.client("bedrock-runtime", region_name="us-east-1", config = config)

In [15]:
# Set the model ID, e.g., Llama 3 70b Instruct.
model_id = "arn:aws:bedrock:us-east-1:794038231401:imported-model/o86ksgjlukpi"

# Define the prompt for the model.
prompt = "Describe the purpose of a 'hello world' program in one line."

# Embed the prompt in Llama 3's instruction format.
formatted_prompt = f"""
<|begin_of_text|><|start_header_id|>user<|end_header_id|>
{prompt}
<|eot_id|>
<|start_header_id|>assistant<|end_header_id|>
"""

# Format the request payload using the model's native structure.
native_request = {
    "prompt": formatted_prompt,
    "max_gen_len": 512,
    "temperature": 0.5,
}

# Convert the native request to JSON.
request = json.dumps(native_request)

try:
    # Invoke the model with the request.
    response = client.invoke_model(modelId=model_id, body=request)

except (ClientError, Exception) as e:
    print(f"ERROR: Can't invoke '{model_id}'. Reason: {e}")
    exit(1)

# Decode the response body.
model_response = json.loads(response["body"].read())

# Extract and print the response text.
response_text = model_response["generation"]
print(response_text)

The 'hello world' program is a simple computer program that outputs the phrase "Hello, World!" to the user.

